# Numerical Variables
# Dealing with missing values
There are many ways data can end up with missing values.
Most machine learning libraries (including scikit-learn) give an error if you try to build a model using data with
missing values.

1)  A simple option: Drop columns or rows with missing values

    Unless most values in the dropped columns are missing, the model loses access to a lot of (potentially useful!)
    information with this approach. As an extreme example, consider a dataset with 10,000 rows, where one important
    column is missing a single entry. This approach would drop the column entirely!


2) A better option: Imputation

    Imputation fills in the missing values with some number. For instance, we can fill in the mean value along each
    column.

    The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would
    get from dropping the column entirely.

3) An Extension to Imputation

    Imputation is the standard approach, and it usually works well. However, imputed values may be systematically
    above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be
    unique in some other way. In that case, your model would make better predictions by considering which values
    were originally missing.

    In this approach, we impute the missing values, as before. And, additionally, for each column with missing
    entries in the original dataset, we add a new column that shows the location of the imputed entries.

    In some cases, this will meaningfully improve results. In other cases, it doesn't help at all.

In [376]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [377]:
iowa_data = pd.read_csv('data/Melbourne_housing_FULL.csv')

In [378]:
print(iowa_data.columns)

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')


In [379]:
features = ['Rooms', 'Bathroom', 'Car', 'Longtitude', 'Lattitude', 'Landsize']
X = iowa_data[features]
y = iowa_data['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=37)

# Define Function to Measure Quality of Each Approach
We define a function score_dataset() to compare different approaches to dealing with missing values. This takes in the
reduced feature matrices and returns the mean absolute error (MAE) from a random forest model.

The purpose is to see, which features have high impact on the accurate target predictions.

To achieve different results, we train our model on different feature matrices, while maintaining the target matrix's
size unchanged.

In [380]:
def mae(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(random_state=17)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

# Approach 1: Drop columns with missing values
This approach will not work for me, because I have NaN's in the Price columns. I will drop rows where the Nan's are
present.

In [381]:
from pandas.api.types import is_numeric_dtype

In [382]:
all_numeric = [column_name for column_name in iowa_data if is_numeric_dtype(iowa_data[column_name])]
print(all_numeric)
print(features)

['Rooms', 'Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']
['Rooms', 'Bathroom', 'Car', 'Longtitude', 'Lattitude', 'Landsize']


# Approach 1: Drop columns or rows
I will drop rows with missing values
Remember that the reduced dataframe must contain the same rows in features and 'Price' columns!

All features are feature and target columns together.

In [383]:
all_features = features + ['Price']

In [384]:
reduced_iowa_data = iowa_data[all_features].dropna(axis=0, how='any')
reduced_X = reduced_iowa_data[features]
reduced_y = reduced_iowa_data['Price']

The idea behind the underlaying operations is to have the same training and validation sets as the default X and y,
except rows in feature and target columns, which contain Nans at different indexes. That's why we need to delete the
entries with the same index (row) in both train/test X and y in order to maintain the proper shape of matrices.

In [385]:
reduced_train_indices = [index for index, row in X_train.iterrows() if index not in reduced_X.index]
reduced_X_train = X_train.copy().drop(reduced_train_indices, axis=0)
reduced_y_train = y_train.copy().drop(reduced_train_indices)

In [386]:
reduced_test_indices = [index for index, row in X_test.iterrows() if index not in reduced_X.index]
reduced_X_test = X_test.copy().drop(reduced_test_indices, axis=0)
reduced_y_test = y_test.copy().drop(reduced_test_indices)

In [387]:
if all(reduced_X_train.index == reduced_y_train.index):
    print('Indexes of reduced train sets are the same - necessary condition fulfilled.')
if all(reduced_X_test.index == reduced_y_test.index):
    print('Indexes of reduced test sets are the same - necessary condition fulfilled.')

Indexes of reduced train sets are the same - necessary condition fulfilled.
Indexes of reduced test sets are the same - necessary condition fulfilled.


In [388]:
print('Mean Absolute Error for Approach 1:\n',
      mae(reduced_X_train, reduced_X_test, reduced_y_train, reduced_y_test))

Mean Absolute Error for Approach 1:
 172810.41165174177


# Approach 2: Imputation

In [389]:
from sklearn.impute import SimpleImputer

In [390]:
imputer = SimpleImputer(strategy='mean')
imputed_iowa_data = pd.DataFrame(data=imputer.fit_transform(iowa_data[all_features]), columns=all_features)
imputed_X = imputed_iowa_data[features]
imputed_y = imputed_iowa_data['Price']

Remember to use fit_transform with the train sets, and only transform with test sets.

In [391]:
imputed_X_train = pd.DataFrame(data=imputer.fit_transform(X_train), columns=X_train.columns)
imputed_X_test = pd.DataFrame(data=imputer.transform(X_test), columns=X_test.columns)
imputed_y_train = pd.DataFrame(data=imputer.fit_transform(y_train.values.reshape(-1, 1)), columns=['Price'])
imputed_y_test = pd.DataFrame(data=imputer.transform(y_test.values.reshape(-1, 1)), columns=['Price'])

In [392]:
print('Mean Absolute Error for Approach 2:\n',
      mae(imputed_X_train, imputed_X_test, imputed_y_train, imputed_y_test))

D:\Anaconda\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Mean Absolute Error for Approach 2:
 261683.4368014598


Although it's simple, filling in the mean value generally performs quite well (but this varies by dataset). While
statisticians have experimented with more complex ways to determine imputed values (such as regression imputation,
for instance), the complex strategies typically give no additional benefit once you plug the results into sophisticated
machine learning models.

# Approach 3: An Extension to Imputation

Firstly check for column names (all_features=feature columns + target column), which contain Nans.

In [393]:
columns_with_nans = [column_name for column_name in iowa_data[features] if any(iowa_data[column_name].isnull())]
print(columns_with_nans)

['Bathroom', 'Car', 'Longtitude', 'Lattitude', 'Landsize']


Create a copy of DataFrame, so you do not overwrite original DataFrame

In [394]:
imputed_X_train_extended = X_train.copy()
imputed_X_test_extended = X_test.copy()
imputed_y_train_extended = y_train.copy()
imputed_y_test_extended = y_test.copy()

Create columns with names COLUMN_WAS_NAN, meaning that a column previously contained at least one NaN, and values in
corresponding columns will be imputed/deleted.

In [395]:
for column_name in columns_with_nans:
    imputed_X_train_extended[column_name + '_was_NaN'] = imputed_X_train_extended[column_name].isnull()
    imputed_X_test_extended[column_name + '_was_NaN'] = imputed_X_test_extended[column_name].isnull()

In [396]:
imputed_y_train_extended = pd.DataFrame(data=y_train, columns=['Price'])
imputed_y_train_extended['Price_was_NaN'] = imputed_y_train_extended['Price'].isnull()

In [397]:
imputed_y_test_extended = pd.DataFrame(data=y_test, columns=['Price'])
imputed_y_test_extended['Price_was_NaN'] = imputed_y_test_extended['Price'].isnull()

In [398]:
imputed_X_train_extended = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_test_extended = pd.DataFrame(imputer.transform(X_test))

In [399]:
imputed_y_train_extended = pd.DataFrame(imputer.fit_transform(y_train.values.reshape(-1, 1)))
imputed_y_test_extended = pd.DataFrame(imputer.fit_transform(y_test.values.reshape(-1, 1)))

In [400]:
print('Mean Absolute Error for Approach 3:\n',
      mae(imputed_X_train_extended, imputed_X_test_extended, imputed_y_train_extended, imputed_y_test_extended))

D:\Anaconda\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Mean Absolute Error for Approach 3:
 261622.55780141085


# Categorical Variables
A categorical variable takes only a limited number of values.

# Dealing with categorical variables

# Approaches:
    1) Drop Categorical variables

        The easiest approach to dealing with categorical variables is to simply remove them from the dataset. This
        approach will only work well if the columns did not contain useful information.

    2) Label encoding

        Label encoding assigns each unique value to a different integer.
        This approach assumes an ordering of the categories:

        "Never" (0) < "Rarely" (1) < "Most days" (2) < "Every day" (3).

        This assumption makes sense in this example, because there is an indisputable ranking to the categories. Not all
        categorical variables have a clear ordering in the values, but we refer to those that do as ordinal variables.
        For tree-based models (like decision trees and random forests), you can expect label encoding to work well with
        ordinal variables.

     3) One-Hot Encoding

        One-hot encoding creates new columns indicating the presence (or absence) of each possible value in the
        original data. To understand this, we'll work through an example.

        Color: red, red, yellow, green, yellow -> red: 1 1 0 0 0, yellow: 0 0 1 0 1, green: 0 0 0 1 0

        In contrast to label encoding, one-hot encoding does not assume an ordering of the categories. Thus, you can
        expect this approach to work particularly well if there is no clear ordering in the categorical data (e.g.,
        "Red" is neither more nor less than "Yellow"). We refer to categorical variables without an intrinsic ranking
        as nominal variables.

        One-hot encoding generally does not perform well if the categorical variable takes on a large number of values
        (i.e., you generally won't use it for variables taking more than 15 different values).

In [401]:
iowa_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


# Approach 1: Drop categorical columns - exclude data of type 'object'.
Non-numeric data types are classified as 'object' dtypes.

In [402]:
all_iowa_reduced = iowa_data.dropna(axis=0, how='any')
all_X_reduced = all_iowa_reduced.loc[:, iowa_data.columns != 'Price']
all_y_reduced = all_iowa_reduced['Price']
all_X_train_reduced, all_X_test_reduced, all_y_train_reduced, all_y_test_reduced = train_test_split(
    all_X_reduced,
    all_y_reduced,
    random_state=33)

In [403]:
num_X_train = all_X_train_reduced.select_dtypes(exclude=['object'])
num_X_test = all_X_test_reduced.select_dtypes(exclude=['object'])

In [404]:
print('Mean Absolute Error for Approach 1:\n',
      mae(num_X_train, num_X_test, all_y_train_reduced, all_y_test_reduced))

Mean Absolute Error for Approach 1:
 172663.12860636064


# Approach 2: Use label encoding

In [405]:
from sklearn.preprocessing import LabelEncoder

In [406]:
label_encoder = LabelEncoder()
label_X_train = all_X_train_reduced.copy()
label_X_test = all_X_test_reduced.copy()
label_y_train = all_y_train_reduced.copy()
label_y_test = all_y_test_reduced.copy()

Find columns that contain non-numeric data types.

In [407]:
categorical_columns = [col for col in all_X_reduced if not is_numeric_dtype(all_X_reduced[col])]

In X_train and X_test there may be different labels. We fit the training data, but not the test data. The labels, that
are in test data, must also be included in train data!

In [408]:
def clean_train_test_labels(label_X_train, label_X_test, label_y_test, categorical_columns):
    for col in label_X_train[categorical_columns]:
        unique_labels_in_column = label_X_train[col].unique()
        for index, val in label_X_test[col].items():
            # Check if label from X_test is in X_train
            if val not in unique_labels_in_column:
                # If label is not present in training data, delete that row from both feature and target test sets.
                label_X_test = label_X_test.drop(index, axis=0)
                label_y_test = label_y_test.drop(index)
    return label_X_test, label_y_test

In [409]:
label_X_test, label_y_test = clean_train_test_labels(
    label_X_train,
    label_X_test,
    label_y_test,
    categorical_columns)

If label encoding will work, it means that all labels from X_test are in X_train.
Otherwise, clean up the X_test.

In [410]:
for col in [col for col in all_X_reduced if not is_numeric_dtype(all_X_reduced[col])]:
    label_X_train[col] = label_encoder.fit_transform(label_X_train[col])
    label_X_test[col] = label_encoder.transform(label_X_test[col])

In [411]:
print('Mean Absolute Error for Approach 2:\n',
      mae(label_X_train, label_X_test, label_y_train, label_y_test))

Mean Absolute Error for Approach 2:
 200464.45642857143


# Approach 3

In [412]:
from sklearn.preprocessing import OneHotEncoder

In [413]:
object_nunique = list(map(lambda col: all_X_reduced[col].nunique(), categorical_columns))
print(object_nunique)

[315, 8764, 3, 5, 250, 77, 33, 8]


OneHotEncoder removes index, so remember to put it back!

In [414]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
oh_X_train_cols_1 = pd.DataFrame(
    data=one_hot_encoder.fit_transform(all_X_train_reduced[categorical_columns]),
    index=all_X_train_reduced.index
)
oh_X_test_cols_1 = pd.DataFrame(
    data=one_hot_encoder.transform(all_X_test_reduced[categorical_columns]),
    index=all_X_test_reduced.index,
)

Remove categorical columns (will replace with one-hot encoding)

In [415]:
oh_num_X_train_cols_1 = all_X_train_reduced.drop(categorical_columns, axis=1)
oh_num_X_test_cols_1 = all_X_test_reduced.drop(categorical_columns, axis=1)

Add one-hot encoded columns to numerical train, test sets.

In [416]:
oh_X_train_1 = pd.concat([oh_num_X_train_cols_1, oh_X_train_cols_1], axis=1)
oh_X_test_1 = pd.concat([oh_num_X_test_cols_1, oh_X_test_cols_1], axis=1)

In [417]:
print('Mean Absolute Error for Approach 3:\n',
      mae(oh_X_train_1, oh_X_test_1, all_y_train_reduced, all_y_test_reduced))

Mean Absolute Error for Approach 3:
 168834.8643249325


It is better not to use hot encoding if number of cardinals (unique entries) in the column exceeds 15.

Create dictionary with creadentials and corresponding columns.
Firstly map the numbers of unique entries in all categorical columns.
Then zip them only if number of entries is less than 15.

In [418]:
unique_categorical = list(map(lambda col: all_X_reduced[col].nunique(), categorical_columns))
credentials = dict((key, val) for key, val in zip(
    all_X_reduced[categorical_columns], unique_categorical) if val <= 15)
columns_to_hot_encode = credentials.keys()

In [419]:
oh_X_train_cols_2 = pd.DataFrame(
    data=one_hot_encoder.fit_transform(all_X_train_reduced[columns_to_hot_encode]),
    index=all_X_train_reduced.index
)
oh_X_test_cols_2 = pd.DataFrame(
    data=one_hot_encoder.fit_transform(all_X_test_reduced[columns_to_hot_encode]),
    index=all_X_test_reduced.index
)

In [420]:
oh_num_X_train_cols_2 = all_X_train_reduced.drop(categorical_columns, axis=1)
oh_num_X_test_cols_2 = all_X_test_reduced.drop(categorical_columns, axis=1)

In [421]:
oh_X_train_2 = pd.concat([oh_num_X_train_cols_2, oh_X_train_cols_2], axis=1)
oh_X_test_2 = pd.concat([oh_num_X_test_cols_2, oh_X_test_cols_2], axis=1)

In [422]:
print('Mean Absolute Error for Approach 3.1:\n',
      mae(oh_X_train_2, oh_X_test_2, all_y_train_reduced, all_y_test_reduced))

Mean Absolute Error for Approach 3.1:
 169864.38546268913
